In [72]:
import networkx as nx
import shapely.wkt as wkt

graph=nx.read_shp("./boundary_shapefile/boundary.shp")

In [73]:
p1=(104.08143, 30.69705)
# p2=(104.08145, 30.69757)
p2=(104.03855, 30.71273)

In [74]:
# https://stackoverflow.com/questions/33078907/get-all-edges-linked-to-a-given-node-in-a-networkx-graph
print(graph.in_edges(p1))
print(graph.out_edges(p1))
print(graph.get_edge_data((104.08139, 30.69641), (104.08143, 30.69705)))

[((104.08139, 30.69641), (104.08143, 30.69705))]
[((104.08143, 30.69705), (104.08145, 30.69757))]
{'FID': 479, 'ShpName': 'boundary', 'Wkb': b'\x00\x00\x00\x00\x02\x00\x00\x00\x02@Z\x055~g\x0e,@>\xb2G\xec\xfe\x9b|@Z\x056&,\xbas@>\xb2q\xdei\xadC', 'Wkt': 'LINESTRING (104.08139 30.69641,104.08143 30.69705)', 'Json': '{ "type": "LineString", "coordinates": [ [ 104.08139, 30.69641 ], [ 104.08143, 30.69705 ] ] }'}


In [75]:
def combine_nodes(graph, p1, p2):
    # https://stackoverflow.com/questions/40674597/networkx-add-node-seems-to-ignore-attributes-parsed-as-dictionary
    # global graph

    # if not p1 in graph.nodes or not p2 in graph.nodes:
    #     return p1

    new_node=((p1[0]+p2[0])/2, (p1[1]+p2[1])/2)
    graph.add_node(new_node)

    for pair in graph.out_edges(p1):
        edge=graph.get_edge_data(pair[0], pair[1])
        if pair[1]!=p2:
            graph.add_edge(new_node, pair[1], **edge)

    for pair in graph.in_edges(p1):
        edge=graph.get_edge_data(pair[0], pair[1])
        if pair[0]!=p2:
            graph.add_edge(pair[0], new_node, **edge)

    for pair in graph.out_edges(p2):
        edge=graph.get_edge_data(pair[0], pair[1])
        if pair[1]!=p1:
            graph.add_edge(new_node, pair[1], **edge)

    for pair in graph.in_edges(p2):
        edge=graph.get_edge_data(pair[0], pair[1])
        if pair[0]!=p1:
            graph.add_edge(pair[0], new_node, **edge)

    graph.remove_node(p1)
    graph.remove_node(p2)

    # print("nodes = {}".format(len(graph.nodes)))

    return new_node

In [76]:
g=nx.DiGraph()
g.add_edge((1, 1), (2, 2), weight=1)
g.add_edge((2, 2), (3, 3), weight=2)
g.add_edge((3, 3), (1, 1), weight=3)

combine_nodes(g, (1, 1), (2, 2))

(1.5, 1.5)

In [77]:
print(g.in_edges((1.5, 1.5)))
print(g.out_edges((1.5, 1.5)))

print(g.get_edge_data((1.5, 1.5), (3, 3)))
print(g.get_edge_data((3, 3), (1.5, 1.5)))

print(len(g.nodes))

[((3, 3), (1.5, 1.5))]
[((1.5, 1.5), (3, 3))]
{'weight': 2}
{'weight': 3}
2


In [78]:
def get_manhattan_distance(p1, p2):
    return abs(p1[0]-p2[0])+abs(p1[1]-p2[1])

In [79]:
clusters=[]
nodes=list(graph.nodes)
for i in range(len(nodes)):
    cluster=[nodes[i]]
    for j in range(i+1, len(nodes)):
        if get_manhattan_distance(nodes[i], nodes[j])<0.0003:
            cluster.append(nodes[j])
    clusters.append(cluster)

for cluster in clusters:
    node=cluster[0]
    if not node in graph.nodes:
        continue
    for j in range(1, len(cluster)):
        try:
            if not cluster[j] in graph.nodes:
                continue
            node=combine_nodes(graph, node, cluster[j])
        except nx.NetworkXError:
            pass

In [80]:
nx.write_gpickle(graph, "./road_graph.gpickle")

In [81]:
g=nx.read_gpickle("./road_graph.gpickle")
graph=nx.read_shp("./boundary_shapefile/boundary.shp")

print(len(g.nodes))
print(len(graph.nodes))

4248
8335
